In [1]:
from torch import FloatTensor
from torch import LongTensor

In [2]:
class Module(object):
    def forward(self, *input): 
        raise NotImplementedError
    def backward(self, *gradwrtoutput): 
        raise NotImplementedError
    def param(self): 
        return []

In [3]:
class LossCrossEntropy(Module):
    def forward(self, x_in, target): 
        self.last_input = x_in
        self.last_target = target
        temp1 = x_in.gather(1, target.view(-1,1)).exp().squeeze()
        temp2 = x_in.exp().sum(1)
        return (-1.0/x_in.size()[0])*((temp1/temp2).log().sum())
    def backward(self):
        temp1 = self.last_input.exp().sum(1).view(-1,1)
        temp2 = -1*self.last_input.exp()/temp1
        j = LongTensor(list(range(self.last_input.size(0))))
        temp2[j, self.last_target] = temp2[j, self.last_target] + 1
        return (-1.0/self.last_input.size()[0])*temp2

In [4]:
from torch.autograd import Variable
a = FloatTensor([[1, 9, 6], [4, 5, 3]])
b = LongTensor([1, 2])
a_var = Variable(a, requires_grad=True)
b_var = Variable(b)

In [5]:
from torch import nn
test1 = nn.CrossEntropyLoss()
test2 = LossCrossEntropy()

In [6]:
res = test1(a_var, b_var)
res2 = test2.forward(a,b)
print(res.data[0])
print(res2)

1.228256344795227
1.2282563876360655


In [7]:
res.backward()
a_var.grad.data


 0.0002 -0.0239  0.0237
 0.1224  0.3326 -0.4550
[torch.FloatTensor of size 2x3]

In [8]:
test2.backward()


 0.0002 -0.0239  0.0237
 0.1224  0.3326 -0.4550
[torch.FloatTensor of size 2x3]